In [5]:
#main code
import csv
import pandas as pd
from sgp4.api import Satrec
from sgp4 import omm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
import requests
import time




def fetch_and_save_data(api_url, output_file):
    try:
        response = requests.get(api_url)
        
        if response.status_code == 200:
            data = response.json()

            with open(output_file, 'w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=data[0].keys())
                writer.writeheader()
                for item in data:
                    writer.writerow(item)

            print(f"Дані оновлені та записані в{output_file}")
        else:
            print(f"Помилка отримання даних. Статус коду: {response.status_code}")

    except Exception as e:
        print(f"Виникла помилка: {str(e)}")

api_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=last-30-days&FORMAT=json"
output_file = "db.csv"
fetch_and_save_data(api_url, output_file)



def count_lines_in_file(file_path):
    line_count = 0
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line_count += 1
    return line_count



file_path = 'db.csv'  
result = count_lines_in_file(file_path) - 1


data = pd.read_csv('db.csv')


with open('db.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

R = []



for i in range(result):
    field = fields[i]
    sat = Satrec("2023-172B")
    satellite = omm.initialize(sat, field)
    jd, fr = 2460326, 0.8473379629629629
    e, r, v = sat.sgp4(jd, fr)
    R.append(r)
    




data = np.array(R)
data = data[~np.isnan(data).any(axis=1)]

fig = go.Figure(data=[go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers')])
fig.show()
 


model = DBSCAN(eps=100, min_samples=2)
model.fit_predict(data)
pred = model.fit_predict(data)

fig = go.Figure(data=[go.Scatter3d(x=data[:,0], y=data[:,1], z=data[:,2],
                                   mode='markers', marker={"size":12, "color":pred})])

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

Помилка отримання даних. Статус коду: 403


EmptyDataError: No columns to parse from file